**Proyek Mini: Analisis Perilaku Pengguna**

**Konteks: Bayangkan Anda adalah seorang Data Analyst di sebuah perusahaan aplikasi mobile. Anda diberi dataset (Anda bisa membuat data tiruan sendiri) yang berisi informasi pengguna: user_id, age_category ('Di Bawah 25', 'Di Atas 25'), dan action_taken ('Membeli Produk', 'Tidak Membeli').**

----

In [2]:
import pandas as pd

user_df = pd.read_csv('analisis_pengguna.csv')
user_df.head()

,user_id,age_category,action_taken
0,1,Di Atas 25,Membeli Produk
1,2,Di Bawah 25,Tidak Membeli
2,3,Di Atas 25,Tidak Membeli
3,4,Di Atas 25,Tidak Membeli
4,5,Di Bawah 25,Membeli Produk


**TASK**

1. Uji Proporsi 1 Sampel:
- Hipotesis: Tim produk mengklaim bahwa 40% dari semua pengguna aplikasi berusia 'Di Bawah 25'. Ujilah klaim ini.
- Langkah: Hitung proporsi sampel ($\hat{p}$), tentukan proporsi hipotesis ($p_0=0.4$), dan hitung z-score serta p-value seperti pada materi.

In [6]:
# One-Sample Proportion merupakan Test yang bertujuan untuk menguji klaim mengenai proporsi populasi
import numpy as np 

# proporsi sampel p_hat
p_hat = (user_df['age_category'] == 'Di Bawah 25').mean()
print('proporsi pengguna', p_hat)

# proporsi hipotesis (p_0)
p_0 = 0.4

# Hitung ukuran sampel (n)
n = len(user_df)
print('ukuran sample', n)

# hitung nila z-score

numerator = p_hat - p_0 
denominator = np.sqrt(p_0 * (1 - p_0) / n) 
z_score = numerator / denominator 
print('z-score:', z_score)

proporsi pengguna 0.5
ukuran sample 30
z-score: 1.1180339887498947


---
2. Uji Proporsi 2 Sampel:
- Hipotesis: Selidiki apakah ada perbedaan signifikan dalam proporsi pembelian produk antara pengguna 'Di Bawah 25' dan 'Di Atas 25'.
-  Langkah: Buat hipotesis nol ($H_0: p_{<25} - p_{\ge25} = 0$). Gunakan fungsi proportions_ztest dari statsmodels untuk menghitung z-score dan p-value.

---
3. Uji Chi-Square Independensi (Opsional, jika ada data tambahan):

- Hipotesis: Jika Anda memiliki data tambahan seperti satisfaction_level ('Puas', 'Netral', 'Tidak Puas'), ujilah apakah ada hubungan antara age_category dan satisfaction_level.
- Langkah: Gunakan fungsi pingouin.chi2_independence untuk melakukan pengujian.